## Q1.

Add a __setitem__ to the python linked list implementation from the lecture (this past wednesday).

In [1]:
#your code here

def __setitem__(self, key, c):
    self.insert((key,c))

## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [2]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [3]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [4]:
g = make_data(5, 10)
list(g)

[1000000000.8007792,
 1000000001.099089,
 1000000005.6191047,
 1000000002.3382626,
 999999994.9325846,
 1000000002.6687838,
 1000000000.3893675,
 1000000003.1991405,
 1000000001.5009574,
 999999995.4134774,
 999999997.8654048]

In [5]:
g = online_mean(make_data(5, 100))
print(type(g))
list(g)

<class 'generator'>


[1000000000.0383301,
 999999999.8608612,
 1000000001.9833528,
 1000000002.0076723,
 1000000001.3045043,
 1000000001.6368686,
 1000000001.2049081,
 1000000000.9359994,
 1000000000.8467113,
 1000000000.7816591,
 1000000000.7046827,
 1000000000.5493864,
 1000000000.7307124,
 1000000000.7744975,
 1000000000.5426629,
 1000000000.3482492,
 1000000000.3119509,
 1000000000.7185297,
 1000000000.773973,
 1000000000.7601937,
 1000000000.730617,
 1000000000.6295277,
 1000000000.6185038,
 1000000000.5935628,
 1000000000.324125,
 1000000000.4020188,
 1000000000.4837278,
 1000000000.5584495,
 1000000000.6277989,
 1000000000.6275498,
 1000000000.625016,
 1000000000.5676911,
 1000000000.6185404,
 1000000000.6680492,
 1000000000.7856728,
 1000000000.7785002,
 1000000000.7797543,
 1000000000.899072,
 1000000001.0228016,
 1000000000.9715472,
 1000000000.8380038,
 1000000000.9180142,
 1000000000.7241408,
 1000000000.727064,
 1000000000.7324262,
 1000000000.8321835,
 1000000000.6961212,
 1000000000.7164052,

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [26]:
def online_mean_dev(iterator):
    #initialize
    std = 0
    m = 0
    n = 0
    #iterate through the iterator
    for item in iterator:
        m_0 = m
        n = n + 1
        m = m + ((item - m)/n)
        
        if n > 1:
            std = ((std + ((item - m_0)*(item - m))) / (n - 1))**0.5
            yield (n, item, m, std)


Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [27]:
data_with_stats = online_mean_dev(make_data(5, 100000))

## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.

In [28]:
def is_ok(level, t):
    #Initialize
    item = t[1]
    m = t[2]
    std = t[3]
    #Update
    s = level - std
    left_bound = m - s
    right_bound = m + s
    
    if(item <= right_bound) and (item >= left_bound):
        yield True
    else:
        yield False

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [29]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [30]:
list(anomalies)#materialize

[]

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).